Importation des library

In [ ]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
import anywidget
import os
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import talib
plt.style.use('dark_background')

Chargement des données

In [ ]:
symbol = ['BTCUSDT']
time = "1m"
chemin = r'C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1m\ETHUSDT\ETHUSDT_1.8d_2026-01-04_TO_2026-01-04_1m'
data = pd.read_parquet(chemin)

In [ ]:
# Separation des données en train et test (IS/OOS) (70/30%)
split_idx = int(0.9*len(data))
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

# Affichage de la séparation des données
plt.figure(figsize=(10, 1))
plt.barh(['Dataset'], [len(data)], color='lightgray', alpha=0.3)
plt.barh(['Dataset'], [split_idx], color='blue', alpha=0.5, label='Train')
plt.barh(['Dataset'], [len(data)-split_idx], left=split_idx, color='orange', alpha=0.5, label='Test')
plt.title('Train/Test Split Visualization')
plt.legend()
plt.show()

In [ ]:
data

In [ ]:
df = data.copy()

close = df['Close']
open = df['Open']
high = df['High']
low = df['Low']

# =============================
# PARAMETERS
# =============================
RSI_LENGTH = 8
RSI_OVERSOLD = 25
RSI_OVERBOUGHT = 75

STOCH_LENGTH = 14
K_LENGTH = 3
D_LENGTH = 3

RVI_LENGTH = 10

MACD_FAST = 8
MACD_SLOW = 21
MACD_SIGNAL= 8

# =============================
# RISK PARAMETERS
# =============================
FEE = 0.04        # 0.04% (Binance taker) sur future usd
# =============================
# INDICATORS
# =============================

# EMA (trend)
#ema9  = vbt.MA.run(close, window=9, ewm=True).ma
#ema20 = vbt.MA.run(close, window=20, ewm=True).ma

# RSI (momentum)
rsi = talib.RSI(close,RSI_LENGTH).round(2)

# Stochastic RSI
rsi_min = rsi.rolling(STOCH_LENGTH).min()
rsi_max = rsi.rolling(STOCH_LENGTH).max()

den = (rsi_max - rsi_min)
stoch_rsi = 100 * (rsi - rsi_min) / den.replace(0, np.nan)

stoch_rsi_k = stoch_rsi.rolling(K_LENGTH).mean().round(2)
stoch_rsi_d = stoch_rsi_k.rolling(D_LENGTH).mean().round(2)

# RVI (force du mouvement)
#close_open = close - open
#high_low = high - low

#rvi = talib.SMA(close_open, RVI_LENGTH) / talib.SMA(high_low, RVI_LENGTH)
#rvi_signal = talib.SMA(rvi, 4)

# MACD
macd, macd_signal, macd_hist = talib.MACD(
    close,
    fastperiod=MACD_FAST,
    slowperiod=MACD_SLOW,
    signalperiod=MACD_SIGNAL
)
macd = macd.round(2)
macd_signal = macd_signal.round(2)
macd_hist = macd_hist.round(4)
                       
# ATR (risk)
#atr = talib.ATR(df['High'], df['Low'], close, ATR_LENGTH)

# =============================
# STRATEGIE
# =============================
long_entries = ( (rsi <= RSI_OVERSOLD) & (close < ema9) )
short_entries = ( (rsi >= RSI_OVERBOUGHT) & (close > ema9) )
# =============================
# SL / TP PRICES
# =============================
long_sl = close - SL_ATR * atr
long_tp = close + TP_ATR * atr

short_sl = close + SL_ATR * atr
short_tp = close - TP_ATR * atr

# =============================
# BACKTEST
# =============================
overBought = rsi >= 85
portfolio = vbt.Portfolio.from_signals(
    close,
    entries=long_entries,
    exits= overBought,
    short_entries=short_entries,
    short_exits=None,
    sl_stop=0.01,
    tp_stop=0.01,
    fees=FEE,
    freq="1m",
    #direction="both",
    accumulate=False,
    init_cash=500
)

# =============================
# GRAPHIQUE
# =============================
'''
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.7, 0.3], subplot_titles=("Prix", "RSI")
)

# Bougies
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=close,
    name="Prix",
    increasing_line_color='rgba(0,200,0,0.6)',
    decreasing_line_color='rgba(200,0,0,0.6)',
    increasing_fillcolor='rgba(0,200,0,0.3)',
    decreasing_fillcolor='rgba(200,0,0,0.3)'
), row=1, col=1)

# Tracer RSI
fig.add_trace(go.Scatter(
    x=df.index,
    y=rsi,
    name="RSI",
    line=dict(color='orange', width=1)
), row=2, col=1)

# Couleur de fond RSI
fig.add_hrect(
    y0=RSI_OVERBOUGHT,
    y1=RSI_OVERSOLD,
    fillcolor="rgba(126, 87, 194, 0.15)",
    line_width=0.50,
    row=2,
    col=1
)
s

# Tracer EMA
#fig.add_trace(go.Scatter(x=df.index, y=ema9,  name="EMA 9",  line=dict(color='purple')),row=1,col=1)
#fig.add_trace(go.Scatter(x=df.index, y=ema20,  name="EMA 20",  line=dict(color='blue')),row=1,col=1)


trades = portfolio.trades.records_readable

# Entrées LONG
fig.add_trace(go.Scatter(
    x=trades[trades['Direction'] == "Long"]['Entry Timestamp'],
    y=trades[trades['Direction'] == "Long"]['Avg Entry Price'],
    mode='markers',
    marker=dict(color='lime', size=9),
    name='Long Entry'
))

# Entrées SHORT
fig.add_trace(go.Scatter(
    x=trades[trades['Direction'] == "Short"]['Entry Timestamp'],
    y=trades[trades['Direction'] == "Short"]['Avg Entry Price'],
    mode='markers',
    marker=dict(color='red', size=9),
    name='Short Entry'
)) b

# Sorties
fig.add_trace(go.Scatter(
    x=trades['Exit Timestamp'],
    y=trades['Avg Exit Price'],
    mode='markers',
    marker=dict(color='white', size=7),
    name='Exit'
))


fig.update_layout(
    title="Titre",
    template="plotly_dark",
    height=800,
    xaxis_rangeslider_visible=False
)

fig.show()
'''

portfolio.stats()




In [ ]:
portfolio.trades.plot(height=800,width=1200).show()

In [12]:
# =============================
RSI_LENGTH = 8
RSI_OVERSOLD = 25
RSI_OVERBOUGHT = 75

STOCH_LENGTH = 14
K_LENGTH = 3
D_LENGTH = 3

RVI_LENGTH = 10

MACD_FAST = 8
MACD_SLOW = 21
MACD_SIGNAL= 8

df = data.copy()

close = df['Close']
open = df['Open']
high = df['High']
low = df['Low']

macd, macd_signal, macd_hist = talib.MACD(
    close,
    fastperiod=MACD_FAST,
    slowperiod=MACD_SLOW,
    signalperiod=MACD_SIGNAL
)
macd = macd.round(2)
macd_signal = macd_signal.round(2)
macd_hist = macd_hist.round(4)
                       



In [13]:
macd_hist.tail(30)

Open time
2026-01-04 20:48:00+01:00   -0.1337
2026-01-04 20:49:00+01:00    0.2360
2026-01-04 20:50:00+01:00    0.3219
2026-01-04 20:51:00+01:00    0.2976
2026-01-04 20:52:00+01:00    0.2317
2026-01-04 20:53:00+01:00    0.1236
2026-01-04 20:54:00+01:00    0.2335
2026-01-04 20:55:00+01:00    0.4216
2026-01-04 20:56:00+01:00    0.5743
2026-01-04 20:57:00+01:00    0.5885
2026-01-04 20:58:00+01:00    0.4318
2026-01-04 20:59:00+01:00    0.2644
2026-01-04 21:00:00+01:00    0.0310
2026-01-04 21:01:00+01:00   -0.1691
2026-01-04 21:02:00+01:00   -0.1673
2026-01-04 21:03:00+01:00   -0.0161
2026-01-04 21:04:00+01:00   -0.0503
2026-01-04 21:05:00+01:00   -0.0241
2026-01-04 21:06:00+01:00   -0.0157
2026-01-04 21:07:00+01:00    0.4398
2026-01-04 21:08:00+01:00    0.5326
2026-01-04 21:09:00+01:00    0.4456
2026-01-04 21:10:00+01:00    0.4864
2026-01-04 21:11:00+01:00    0.5368
2026-01-04 21:12:00+01:00    0.5105
2026-01-04 21:13:00+01:00    0.5776
2026-01-04 21:14:00+01:00    0.4808
2026-01-04 21:15:0